In [1]:
import sys
sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from interpolant.cell_to_points import *
from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5

In [3]:
np.set_printoptions(linewidth=500)

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))

gm.draw_grid()

In [4]:
cell12 = cell.Cell2D(size=(1,1), ll_vertex=(0,2))
cell22 = cell.Cell2D(size=(1,1), ll_vertex=(0,3))

cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))
cell32 = cell.Cell2D(size=(2,2), ll_vertex=(1,2))

In [5]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2, cell12, cell22]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3, cell32]], layer_number=1, index=[1], div_index=[1])

In [6]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [7]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()

In [8]:
ma._vertex_ddof_index

{((0, 0), (0, 0)): (0, 6),
 ((0, 0), (0, 1)): (3, 2),
 ((0, 0), (1, 0)): (12, 9),
 ((0, 0), (1, 1)): (15, 3),
 ((0, 1), (0, 1)): (0, 2),
 ((0, 1), (0, 2)): (3, 18),
 ((0, 1), (1, 1)): (12, 3),
 ((0, 1), (1, 2)): (15, 19),
 ((0, 2), (0, 2)): (0, 18),
 ((0, 2), (0, 3)): (3, 30),
 ((0, 2), (1, 2)): (12, 19),
 ((0, 2), (1, 3)): (15, 31),
 ((0, 3), (0, 3)): (0, 30),
 ((0, 3), (0, 4)): (3, 42),
 ((0, 3), (1, 3)): (12, 31),
 ((0, 3), (1, 4)): (15, 45),
 ((1, 0), (1, 0)): (0, 61),
 ((1, 0), (1, 2)): (3, 54),
 ((1, 0), (3, 0)): (12, 58),
 ((1, 0), (3, 2)): (15, 55),
 ((1, 2), (1, 2)): (0, 54),
 ((1, 2), (1, 4)): (3, 70),
 ((1, 2), (3, 2)): (12, 55),
 ((1, 2), (3, 4)): (15, 71)}

In [10]:
ma._vertex_ddof_index[((0, 3), (0, 4))] = (ma._vertex_ddof_index[((0, 3), (0, 4))][0], ma._vertex_ddof_index[((0, 0), (0, 0))][1])
ma._vertex_ddof_index[((0, 3), (1, 4))] = (ma._vertex_ddof_index[((0, 3), (1, 4))][0], ma._vertex_ddof_index[((0, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (1, 4))] = (ma._vertex_ddof_index[((1, 2), (1, 4))][0], ma._vertex_ddof_index[((1, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (3, 4))] = (ma._vertex_ddof_index[((1, 2), (3, 4))][0], ma._vertex_ddof_index[((1, 0), (3, 0))][1])

In [11]:
edge_mapping = {((0, 4), (1, 4)): ((0, 0), (1, 0)), ((1, 4), (3, 4)): ((1, 0), (3, 0))}
for (cell, edge), dd in ma._edge_ddof_index.items():
    if edge_mapping.get(edge) is not None:
        edge_slave = edge
        edge_prim = edge_mapping[edge]
        prim_cell = [i for i in list(ma._edge_ddof_index.keys()) if i[1] == edge_prim][0][0]
        ma._edge_ddof_index[(cell, edge_slave)] = [(i[0],j[1])
            for i,j in
            zip(ma._edge_ddof_index[(cell, edge_slave)], ma._edge_ddof_index[(prim_cell, edge_prim)])
        ]

In [12]:
grid_domain = GridDomain(domain_size=(3.,2.), integer_size=(3,2))
grid_domain.make_pointwise_index(ma)

In [13]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)


In [17]:
ma._vertex_ddof_index

{((0, 0), (0, 0)): (0, 6),
 ((0, 0), (0, 1)): (3, 2),
 ((0, 0), (1, 0)): (12, 9),
 ((0, 0), (1, 1)): (15, 3),
 ((0, 1), (0, 1)): (0, 2),
 ((0, 1), (0, 2)): (3, 18),
 ((0, 1), (1, 1)): (12, 3),
 ((0, 1), (1, 2)): (15, 19),
 ((0, 2), (0, 2)): (0, 18),
 ((0, 2), (0, 3)): (3, 30),
 ((0, 2), (1, 2)): (12, 19),
 ((0, 2), (1, 3)): (15, 31),
 ((0, 3), (0, 3)): (0, 30),
 ((0, 3), (0, 4)): (3, 6),
 ((0, 3), (1, 3)): (12, 31),
 ((0, 3), (1, 4)): (15, 9),
 ((1, 0), (1, 0)): (0, 61),
 ((1, 0), (1, 2)): (3, 54),
 ((1, 0), (3, 0)): (12, 58),
 ((1, 0), (3, 2)): (15, 55),
 ((1, 2), (1, 2)): (0, 54),
 ((1, 2), (1, 4)): (3, 61),
 ((1, 2), (3, 2)): (12, 55),
 ((1, 2), (3, 4)): (15, 58)}

In [14]:
deep = MA.assemble()

[(0, 2), (1, 20), (2, 21), (3, 18), (4, 0), (5, 24), (6, 25), (7, 16), (8, 1), (9, 26), (10, 27), (11, 17), (12, 3), (13, 22), (14, 23), (15, 19)] [(0, 6), (1, 4), (2, 5), (3, 2), (4, 7), (5, 12), (6, 13), (7, 0), (8, 8), (9, 14), (10, 15), (11, 1), (12, 9), (13, 10), (14, 11), (15, 3)]
{0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 26, 27} {64, 65, 66, 67, 52, 53, 55, 56, 57, 58, 59, 60} [[61, 62, 63, 54]] [[9, 10, 11, 3, 22, 23, 19]]
 0[(0, 30), (1, 40), (2, 41), (3, 6), (4, 28), (5, 48), (6, 49), (7, 7), (8, 29), (9, 50), (10, 51), (11, 8), (12, 31), (13, 46), (14, 47), (15, 9)] [(0, 18), (1, 32), (2, 33), (3, 30), (4, 16), (5, 36), (6, 37), (7, 28), (8, 17), (9, 38), (10, 39), (11, 29), (12, 19), (13, 34), (14, 35), (15, 31)]
{6, 7, 8, 9, 16, 17, 18, 28, 29, 30, 32, 33, 36, 37, 38, 39, 40, 41, 48, 49, 50, 51} {72, 73, 76, 77, 78, 79, 52, 53, 55, 58, 59, 60} [[54, 74, 75, 61]] [[19, 34, 35, 31, 46, 47, 45]]
 5

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [21]:
def partial_diff_test3(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True, scale_factor=1):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der * scale_factor))
    print('Y derivative: {}'.format(y_der * scale_factor))
    print('XX derivative: {}'.format(xx_der * scale_factor))
    print('YY derivative: {}'.format(yy_der * scale_factor))
    print('XY_derivative: {}'.format(xy_der * scale_factor), end='\n\n')

In [22]:
def partial_diff_test2(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]

    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))

    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')
    return (xx_der + yy_der)/2

In [ ]:
mm = np.zeros(MA.half_glob.shape[0])
for i in grid_domain.normal_index.keys():
    print(i, end=' ')
    mm[i] = partial_diff_test2(matrix=MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
MM = csr_matrix(np.diag(-1/mm))

In [ ]:
for i in sorted([81, 82, 83, 84, 85, 70] + [15, 16, 17, 18, 19, 5, 46, 47, 48, 49, 45]):
    print(i, end=' ')
    partial_diff_test(matrix=MM*MA.half_glob, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
2 * (0.007416255387549075 - 0.023370207540036145 + 0.0892035115259431)

In [ ]:
for num,i in enumerate([37, 38, 39, 37]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.assembled,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       #scale_factor=1/w_ld[num] * MA.I_s2b[5][num]
                       scale_factor=1
)

In [25]:
for num,i in enumerate([75]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.assembled,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       #scale_factor=1/w_ld[num] * MA.I_s2b[5][num]
                       scale_factor=1
)

75 X derivative: 0.006469356449353494
Y derivative: -0.9228755362353351
XX derivative: -0.11028251785467383
YY derivative: -1.6407888685159773
XY_derivative: 0.08753195259981056



In [ ]:
2*(-4.486123159394118 + 8.097144560212651 + -3.110781418448933) * 0.03333334

In [ ]:
0.03333334 * 2*(0.30022197830861924 - 0.5398047719572523 + 0.20739103204915854)

In [ ]:
for num,i in enumerate([81, 82, 83, 84, 85, 70]):
    print(i, end=' ')
    partial_diff_test3(matrix=MA.half_glob,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       scale_factor=1/w_ld[num] * MA.I_s2b[5][num]/2
                       
                       #scale_factor=1
    )
    print('norma_val: ', 1/w_ld[num] * MA.I_s2b[5][num]/2)

In [ ]:
print(deep[0][1].T[5].nonzero())
deep[0][1].T[5].toarray().flatten().take(deep[0][1].T[5].nonzero()[1])[1:]

In [ ]:
print(deep[0][1][5].nonzero())
deep[0][1][5].toarray().flatten().take(deep[0][1][5].nonzero()[1])

In [ ]:
MA.I_s2b[5]

In [ ]:
np.dot(np.dot(w_r, MA.I_s2b), np.linalg.inv(w_l))

In [ ]:
MA.I_b2s.T

In [ ]:
c2p = Cell2PointsConverter(orders=[[5],[5,5]])

In [ ]:
w_rd

In [ ]:
w_ld, w_rd = c2p.get_weights()

In [ ]:
w_l, w_r = c2p.get_diag_weights()

In [ ]:
grid_domain.normal_index[82]

In [ ]:
for i in [81, 82, 83, 84, 85, 70]:
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled, point=grid_domain.normal_index[i], pointnum=i, pointdict=grid_domain.pointwise_index)

In [ ]:
0.07337582303605292/ -0.018924022932711504

In [ ]:
f = h5py.File('../datasources/1_10_globatto_matrices.h5')

In [ ]:
sc = f['mass_matrices/base_M_order_5'][()]

In [ ]:
mm_new = np.diag(sc.sum(axis=1))

In [ ]:
mm_new

In [ ]:
init_op = MA.half_glob * linalg.inv(deep[0][1])

In [ ]:
init_op

In [ ]:
MA.I_s2b[5]